In [0]:
# Importing Libraries

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [0]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# Importing libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Input, Dense, Dropout

In [0]:
 # Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile

zipref = zipfile.ZipFile('/content/drive/My Drive/UCI_HAR_Dataset.zip', mode = 'r')
zipref.extractall('/content/')
zipref.close()
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [0]:
# Create a model object

def create_model(n_hidden = 32, dropout = 0.5, n_layers = 1):
  # Initiliazing the sequential model
  model = Sequential()
  model.add(Input(shape = (timesteps, input_dim)))
  
  if n_layers > 1:
    for layer in range(n_layers - 1):
      # Configuring the parameters
      model.add(LSTM(n_hidden, return_sequences = True))
      # Adding a dropout layer
      model.add(Dropout(dropout))
    # Configuring the parameters
    model.add(LSTM(units = n_hidden))
    # Adding a dropout layer
    model.add(Dropout(dropout))  
  else:
    # Configuring the parameters
    model.add(LSTM(units = n_hidden))
    # Adding a dropout layer
    model.add(Dropout(dropout))  
    
  # Adding a dense output layer with sigmoid activation
  model.add(Dense(n_classes, activation='sigmoid'))
  
  # Compiling the model
  model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
  
  return model


# Viewing model summary
model = create_model()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5376      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
##################################################################
###############  Grid Search using Custom Code ##################
##################################################################


from prettytable import PrettyTable
import itertools

# Setting parameters
dropout = [0.3, 0.5, 0.7]
n_hidden = [16, 32, 48]
n_layers = [1, 2]
epochs = [30] # can be changed in later versions
batch_size = [16] # can be changed in later versions



params = {'epochs': [], 'n_layers' : [], 'n_hidden': [], 'dropout': [], 'acc': [0], 'val_acc' : [0]}



for n_layers, n_hidden, dropout, epochs, batch_size in itertools.product(n_layers, n_hidden, dropout, epochs, batch_size):
  model = create_model(n_hidden = n_hidden, dropout = dropout, n_layers = n_layers)
  
  print("For n_layers = {}, n_hidden = {} and dropout = {} ............".format(n_layers, n_hidden, dropout))

  # Training the model
  fit_model = model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,
          verbose = 1)

  
  params['epochs'].append(epochs)
  params['n_layers'].append(n_layers)
  params['n_hidden'].append(n_hidden)
  params['dropout'].append(dropout)
  params['acc'].append(fit_model.history['acc'][-1])
  params['val_acc'].append(fit_model.history['val_acc'][-1])
  
  acc_train = params['acc']
  acc_valid = params['val_acc']
  if acc_valid[-1] > np.max(acc_valid[:-1]):
    model_best = model
    param_best = {'epochs': epochs, 'n_layers' : n_layers, 'n_hidden': n_hidden, 'dropout': dropout,\
                  'acc': acc_train[-1], 'val_acc' : acc_valid[-1]}
    print('Found better model, replacing earlier')
    
  print("With n_layers = {}, n_hidden= {} and dropout = {}, the Train_Acc is = {} and Val_Acc is = {} \n\n".\
        format(n_layers, n_hidden, dropout, acc_train[-1], acc_valid[-1]))

   
    
    
# Display Pretty Table using PrettyTable library
params['acc'].pop(0)
params['val_acc'].pop(0)
df = pd.DataFrame(params)  
DF = PrettyTable()
DF.field_names = df.columns
for row in range(df.count().max()):
  DF.add_row(df.iloc[row])
print(DF)

# Displar best parameters
print("The Best parameters are:", param_best, sep = '\n')
                           

For n_layers = 1, n_hidden = 16 and dropout = 0.3 ............
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 68s 9ms/sample - loss: 1.3962 - acc: 0.3780 - val_loss: 1.3008 - val_acc: 0.4479
Epoch 2/30
7352/7352 [==============================] - 69s 9ms/sample - loss: 1.1406 - acc: 0.4962 - val_loss: 1.0953 - val_acc: 0.5273
Epoch 3/30
7352/7352 [==============================] - 69s 9ms/sample - loss: 0.9093 - acc: 0.5983 - val_loss: 0.9257 - val_acc: 0.5728
Epoch 4/30
7352/7352 [==============================] - 68s 9ms/sample - loss: 0.8573 - acc: 0.6128 - val_loss: 1.1911 - val_acc: 0.5843
Epoch 5/30
7352/7352 [==============================] - 68s 9ms/sample - loss: 0.7727 - acc: 0.6570 - val_loss: 0.8552 - val_acc: 0.6630
Epoch 6/30
7352/7352 [==============================] - 68s 9ms/sample - loss: 0.6640 - acc: 0.7047 - val_loss: 0.7303 - val_acc: 0.6729
Epoch 7/30
7352/7352 [==============================] - 67s 9ms/sam

KeyboardInterrupt: ignored

In [0]:
params

{'acc': [0, 0.94545704, 0.9427367, 0.9442329],
 'dropout': [0.3, 0.5, 0.7],
 'epochs': [30, 30, 30],
 'n_hidden': [16, 16, 16],
 'n_layers': [1, 1, 1],
 'val_acc': [0, 0.9073634, 0.90193415, 0.8880217]}

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, grid.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 512        0        25        0                   0   
SITTING                  3      410        75        0                   0   
STANDING                 0       87       445        0                   0   
WALKING                  0        0         0      481                   2   
WALKING_DOWNSTAIRS       0        0         0        0                 382   
WALKING_UPSTAIRS         0        0         0        2                  18   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            3  
STANDING                           0  
WALKING                           13  
WALKING_DOWNSTAIRS                38  
WALKING_UPSTAIRS                 451  


In [0]:
score = grid.score(X_test, Y_test)

In [0]:
score

[0.3087582236972612, 0.9097387173396675]